# American Option Pricing

class for american option pricing using Least-Squares Method

In [10]:
import numpy as np

class AmericanOptionsLSMC(object):
    """
    S0 : initial stock price
    strike : strike price
    T : time to maturity (1 for a year)
    M : steps
    r : risk-free rate (constant for the program)
    sigma : stock price volatility (constant for the program) 
    
    """
    def __init__(self, option_type, S0, strike, T, M, r, sigma, simulations):
        try:
            self.option_type = option_type
            assert S0 > 0
            self.S0 = float(S0)
            assert strike > 0
            self.strike = float(strike)
            assert T > 0
            self.T = float(T)
            assert M > 0
            self.M = int(M)
            self.r = float(r)
            assert sigma > 0
            self.sigma = float(sigma)
            assert simulations > 0
            self.simulations = int(simulations)
        except ValueError:
            print('Error passing parameters, check the inputs')

        if option_type != 'call' and option_type != 'put':
            raise ValueError("Error: enter 'call' or 'put' for option type")

        self.unit_time = self.T / self.M
        self.discount = np.exp(-self.r * self.unit_time)

    @property
    def stock_price_MC(self, seed = 111):
        np.random.seed(seed)
        stock_price_MC = np.zeros((self.M + 1, self.simulations), dtype=np.float64)
        stock_price_MC[0,:] = self.S0
        for t in range(1, self.M + 1):
            path_brownian = np.random.standard_normal(self.simulations)
            stock_price_MC[t, :] = (stock_price_MC[t - 1, :] 
                                    * np.exp((self.r - self.sigma ** 2 / 2) * self.unit_time
                                    + self.sigma * path_brownian * np.sqrt(self.unit_time)))
        return stock_price_MC

    @property
    def payoff_MC(self):
        if self.option_type == 'call':
            payoff = np.maximum(self.stock_price_MC - self.strike,
                                np.zeros((self.M + 1, self.simulations),dtype=np.float64))
        else:
            payoff = np.maximum(self.strike - self.stock_price_MC,
                                np.zeros((self.M + 1, self.simulations),dtype=np.float64))
        return payoff

    @property
    def option_values_MC(self):
        value_matrix = np.zeros_like(self.payoff_MC)
        value_matrix[-1, :] = self.payoff_MC[-1, :]
        for t in range(self.M - 1, 0 , -1):
            reg = np.polyfit(self.stock_price_MC[t, :], value_matrix[t + 1, :] * self.discount, 2)
            holding_value = np.polyval(reg, self.stock_price_MC[t, :])
            value_matrix[t, :] = np.where(self.payoff_MC[t, :] > holding_value,
                                          self.payoff_MC[t, :],
                                          value_matrix[t + 1, :] * self.discount)
        return value_matrix[1,:] * self.discount

    @property
    def price(self): return np.sum(self.option_values_MC) / self.simulations
    
    @property
    def delta(self):
        diff = self.S0 * 0.01
        diff_option_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.sigma, self.simulations)
        diff_option_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.sigma, self.simulations)
        return (diff_option_1.price - diff_option_2.price) / float(2. * diff)
    
    @property
    def gamma(self):
        diff = self.S0 * 0.01
        diff_option_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.sigma, self.simulations)
        diff_option_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.sigma, self.simulations)
        return (diff_option_1.delta - diff_option_2.delta) / float(2. * diff)

    @property
    def vega(self):
        diff = self.sigma * 0.01
        diff_option_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.sigma + diff, self.simulations)
        diff_option_2 = AmericanOptionsLSMC(self.option_type, self.S0,
                                       self.strike, self.T, self.M, 
                                       self.r, self.sigma - diff, self.simulations)
        return (diff_option_1.price - diff_option_2.price) / float(2. * diff)    
    
    @property
    def rho(self):        
        diff = self.r * 0.01
        diff_option_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                   self.strike, self.T, self.M, 
                                   self.r + diff, self.sigma, self.simulations)
        diff_option_2 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                   self.strike, self.T, self.M, 
                                   self.r - diff, self.sigma, self.simulations)
        return (diff_option_1.price - diff_option_2.price) / float(2. * diff)
 
    @property
    def theta(self): 
        diff = 1 / 252.
        diff_option_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T + diff, self.M, 
                                       self.r, self.sigma, self.simulations)
        diff_option_2 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T - diff, self.M, 
                                       self.r, self.sigma, self.simulations)
        return (diff_option_2.price - diff_option_1.price) / float(2. * diff)


use class defined above to price american option

In [11]:
import datetime
T=(datetime.date(2019,4,18) - datetime.date(2019,1,26)).days / 365.0
#from Stock_Volatility import get_hist_volatility
#sigma=get_hist_volatility('AAPL','2018-01-26','2019-01-26','daily')
sigma=0.2721
S0=157.76
strike=160
M=50
r=0.023

In [12]:
from time import time
start_time = time()

AAPL190418C00165000 = AmericanOptionsLSMC('call',S0, strike, T, M, r, sigma, 10000  )
print( 'Price: ', AAPL190418C00165000.price)
print( 'Delta: ', AAPL190418C00165000.delta) 
print( 'Gamma: ', AAPL190418C00165000.gamma)
print( 'Vega:  ', AAPL190418C00165000.vega)
print( 'Rho:   ', AAPL190418C00165000.rho)
print( 'Theta: ', AAPL190418C00165000.theta)

end_time = time()
run_time = end_time - start_time
print( "running time",run_time)

Price:  6.881338444586481
Delta:  0.4908491787332708
Gamma:  0.019157205262414007
Vega:   28.531823483535494
Rho:    34.84321540169441
Theta:  -18.420675270824283
running time 127.08225655555725
